# IntPic

This is a notebook showing some drawings related to numerical semigroups produced with the package [IntPic](https://www.gap-system.org/Packages/intpic.html).

We start by loading the package and setting the scale to 1.5, so that the pictures show bigger in the notebook.

In [2]:
LoadPackage("IntPic");
IP_TikzDefaultOptionsForArraysOfIntegers.scale:="1.5";;

true

Next we emphasize the "small elements" of the numerical semigroup $\langle 5,7\rangle$.

In [5]:
ns:=NumericalSemigroup(13,15,17);;
tkz := TikzCodeForNumericalSemigroup(ns,["small_elements"]);;
JupyterSplashTikZ(tkz);

We can modify the options to display the semigroup on a table instead.

In [9]:
ns := NumericalSemigroup(5,7);; 
opts:=rec(ns_table := true,negatives:=false);;
tkz := TikzCodeForNumericalSemigroup(ns,["small_elements"],opts);;
JupyterSplashTikZ(tkz);

In [13]:
opts := rec(ns_table:=true,colors:=["blue", "red!70", "-red", "black!40"]);;
highlights := ["conductor", "min_generators", "small_elements"];;
tkz := TikzCodeForNumericalSemigroup(ns,highlights,opts);;
JupyterSplashTikZ(tkz);

In [18]:
ns := NumericalSemigroup(7,11,38,41);;
highlights := ["conductor", "min_generators", "small_elements"];;
options := rec(ns_table:=true,colors:=["blue", "red!70", "-red", "black!40"],negatives:=false);;
tkz := TikzCodeForNumericalSemigroup(ns,highlights,options);;
JupyterSplashTikZ(tkz);